# 2.1 Fundamentos de Window Functions\n\n- Objetivo: usar OVER() para cálculos fila-a-fila y rankings.\n- Prerrequisitos: ventas pobladas en `fact_ventas`.\n- Ejercicios: RANK(), DENSE_RANK(), LAG/LEAD.\n- Reto: variación porcentual robusta evitando división por cero.\n- Errores comunes: olvidar PARTITION BY, ORDER BY incorrecto en LAG/LEAD.

In [ ]:
-- Ranking de productos por unidades vendidas (desc)
SELECT producto_id, SUM(cantidad) AS unidades,
       RANK() OVER (ORDER BY SUM(cantidad) DESC) AS rk
FROM dbo.fact_ventas
GROUP BY producto_id;

In [ ]:
-- Ventas por día y acumulado
SELECT fecha, SUM(cantidad) AS unidades_dia,
       SUM(SUM(cantidad)) OVER (ORDER BY fecha ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS unidades_acumuladas
FROM dbo.fact_ventas
GROUP BY fecha
ORDER BY fecha;

🟢 Ejercicio: Agregar DENSE_RANK por margen total de producto.

In [ ]:
SELECT p.producto_id, p.nombre,
       SUM(fv.cantidad*(p.precio_unitario-p.costo_unitario)) AS margen_total,
       DENSE_RANK() OVER (ORDER BY SUM(fv.cantidad*(p.precio_unitario-p.costo_unitario)) DESC) AS drk
FROM dbo.fact_ventas fv
JOIN dbo.dim_productos p ON fv.producto_id = p.producto_id
GROUP BY p.producto_id, p.nombre;

🟠 Ejercicio: Usar LAG para comparar unidades de cada producto vs día anterior.

In [ ]:
SELECT producto_id, fecha, SUM(cantidad) AS unidades_dia,
       LAG(SUM(cantidad)) OVER (PARTITION BY producto_id ORDER BY fecha) AS unidades_previas
FROM dbo.fact_ventas
GROUP BY producto_id, fecha
ORDER BY producto_id, fecha;

🔴 Reto: Calcular la variación porcentual diaria acumulada de margen usando una combinación de SUM(), LAG y expresión CASE para evitar división por cero.

Errores comunes: olvidar PARTITION BY; mezclar filtros fuera del window sub-optimizing; no ordenar correctamente para LAG/LEAD.